In [24]:
import building_tokenizer as toker
import json
import pandas as pd
from collections import Counter, OrderedDict
from copy import deepcopy
from functools import reduce
from IPython.display import Audio

In [25]:
#naeListDict['Applied_Physics_and_Mathematics'] = ['514', '513', '604']
#naeListDict['Computer_Science_and_Engineering_Building'] = ['505', '506']
#naeListDict['Conrad_Prebys_Music_Center'] = ['523']
#naeListDict['San_Diego_Supercomputer_Center'] = ['668', '575', '524', '520']
#naeListDict['Literature_Building'] = ['553']
#naeListDict['Structural_and_Materials_Engineering_Building'] = ['572', '573', '574']
#naeListDict['Jacobs_Hall'] = ['620', '621', '622', '623', '650', '639', '638', '642', '643', '640', \
#                              '641', '637', '646', '644', '645']
#naeListDict['Warren_Lecture_Hall'] = ['555', '552']
#naeDict = dict()
#naeDict['bonner'] = ["607", "608", "609", "557", "610"]
#naeDict['ap_m'] = ['514', '513','604']
#naeDict['bsb'] = ['519', '568', '567', '566', '564', '565']
#naeDict['ebu3b'] = ["505", "506"]
#naeDict['music'] = ['523']
#naeDict['sme'] = ['572', '573', '574']
#naeDict['otterson'] = ["518", "517", "589", "590"]

In [26]:
token_map_dict = dict()
token_map_dict['ap_m'] = {
    'vavb': ['vav', 'b'],
    'lvfd': ['l', 'vfd'],
    'rvfd': ['r', 'vfd'],
    'bsmtsf': ['bsmt', 'sf'],
    'lvlco': ['lvl', 'co']
}
token_map_dict['ebu3b'] = dict()

token_map = token_map_dict['ap_m']

In [27]:
def word_tokenize(word):
    if token_map.get(word):
        return token_map[word]
    else:
        return [word]
    

In [28]:
buildingNameList = ['ebu3b', 'sme', 'ap_m', 'music']
notUcsdBuildings = ['ghc']

adder = lambda x,y:x+y


for buildingName in buildingNameList:
    for tokenType in ['AlphaAndNum', 'JustSeparate']:

        if not buildingName in notUcsdBuildings:

            bacnetTypeMapDF = pd.DataFrame.from_csv('metadata/bacnettype_mapping.csv')
            unitMap = pd.read_csv('metadata/unit_mapping.csv').set_index('unit')
            for val in Counter(unitMap.keys()).values():
                if val>1:
                    "Unit map file ERROR"
                    assert(False)

            #sensorDF, nameList, jcinameList, descList, unitList, bacnettypeList, wordList = \
            #toker.structure_metadata(buildingName=buildingName, tokenType=tokenType, withDotFlag=False)

            sensorDF, rawNameList, rawJcinameList, rawDescList, _, _, _ =\
            toker.structure_metadata(buildingName=buildingName, tokenType=tokenType, withDotFlag=False)
            #toker.structure_metadata(buildingName=buildingName, tokenType='JustSeparate', withDotFlag=False)

        '''
        else: 
            filename = 'metadata/'+buildingName+'_sensor_types_location.csv'
            #   filename = 'metadata/%s_sensor_types_location.csv'%buildingName
            df = pd.read_csv(filename)
            sentenceDict = dict()    
            for i,raw in enumerate(df['bas_raw'].tolist()):
                sentenceDict[i] = toker.tokenize(tokenType, raw)
        '''     

        #adder = lambda x,y:x+y
        #splitter = lambda x:x.split()
        #bacnetTypes = list(set(reduce(adder,map(splitter,origBacnettypeList),[])))

        ## Common part

        #with open('data/'+buildingName+'_str_score_dict.pkl', 'rb') as fp:
        #    scoreDictDict = pickle.load(fp)
        #    ### If a word is exactly matched with another, fix it.
        #    for word, scoreDict in scoreDictDict.items():
        #        if word in scoreDict.keys():
        #            scoreDictDict[word] = {word:1}
        #    scoreDictDict['co'] = {'co2':1}

        #for bacnetType in bacnetTypes:
        #    scoreDictDict[bacnetType] = {bacnetType:1}
        #units = list(set(unitMap['word'].tolist()))
        #for unit in units:
        #    if type(unit)==str:
        #        scoreDictDict[unit] = {unit:1}
        splitter = lambda x:x.split(' ')
        sentenceDict = dict()
        charSentenceDict = dict()
        for i, row in enumerate(zip(rawNameList, rawJcinameList, rawDescList)):
            para = list()
            for ele in row:
                para.extend(reduce(adder,[word_tokenize(x) for x in ele.split(' ') if x!=''],[]))
                para.append('\n')
            sentenceDict[sensorDF.iloc[i].name] = para
            charSentenceDict[sensorDF.iloc[i].name] = list(reduce(adder, para, ''))
        with open("metadata/%s_sentence_dict_%s.json" % (buildingName, tokenType.lower()), "w") as fp:
            json.dump(sentenceDict, fp, indent=2, separators=(',',':'))
        with open("metadata/%s_char_sentence_dict_%s.json" % (buildingName, tokenType.lower()), "w") as fp:
            json.dump(charSentenceDict, fp, indent=2, separators=(',',':'))

In [29]:
print("Finished!!!")
sound_file = 'etc/fins_success.wav'
Audio(url=sound_file, autoplay=True)

Finished!!!
